In [1]:
import pandas as pd
import os
import gc

from bokeh.io import output_notebook, show

output_notebook()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import bokeh
bokeh.__version__

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.dao.csv_dao import load_user_gps_csv
from src.dao.dbdao import DBDAO
from src.plot.plot import plot_user_loc

Loading BokehJS ...

working dir /home/tales/dev/master/mdc_analysis


In [9]:
def box_limits(users):
    global_min_lon = 999999999
    global_min_lat = 999999999
    global_max_lat = -999999999
    global_max_lon = -999999999
    
    for userid in users:
        user_gps_data = load_user_gps_csv(userid=userid)
        if len(user_gps_data) == 0:
            continue
            
        user_min_lat = user_gps_data["latitude"].min()
        user_min_lon = user_gps_data["longitude"].min()
        user_max_lat = user_gps_data["latitude"].max()
        user_max_lon = user_gps_data["longitude"].max()
        
        if user_min_lat < global_min_lat:
            global_min_lat = user_min_lat
            
        if user_max_lat > global_max_lat:
            global_max_lat = user_max_lat
        
        if user_min_lon < global_min_lon:
            global_min_lon = user_min_lon
            
        if user_max_lon > global_max_lon:
            global_max_lon = user_max_lon
            
    return {"min_lon": global_min_lon, "max_lon": global_max_lon,
            "min_lat": global_min_lat, "max_lat": global_max_lat}

def bounding_box(users, expand=None):
    limits = box_limits(users)
    
    p3 = {"latitude": limits["min_lat"], "longitude": limits["min_lon"]}
    p4 = {"latitude": limits["min_lat"], "longitude": limits["max_lon"]}
    p2 = {"latitude": limits["max_lat"], "longitude": limits["min_lon"]}
    p1 = {"latitude": limits["max_lat"], "longitude": limits["max_lon"]}
    
    if not expand is None:
        p1["longitude"] = p1["longitude"] + abs(p1["longitude"] - p2["longitude"]) * expand
        p1["latitude"]  = p1["latitude"]  + abs(p1["latitude"]  - p4["latitude"])  * expand

        p2["longitude"] = p2["longitude"] - abs(p1["longitude"] - p2["longitude"]) * expand
        p2["latitude"]  = p2["latitude"]  + abs(p2["latitude"]  - p3["latitude"])  * expand

        p3["longitude"] = p3["longitude"] - abs(p4["longitude"] - p3["longitude"]) * expand
        p3["latitude"]  = p3["latitude"]  - abs(p2["latitude"]  - p3["latitude"])  * expand

        p4["longitude"] = p4["longitude"] + abs(p4["longitude"] - p3["longitude"]) * expand
        p4["latitude"]  = p4["latitude"]  - abs(p1["latitude"]  - p4["latitude"])  * expand
    
    return pd.DataFrame([p1, p2, p3, p4])

In [3]:
users = DBDAO().users_list(drop_test_users=True)

In [4]:
box = bounding_box(users)
box

,latitude,longitude
0,47.828,10.486
1,47.828,5.874
2,45.812,10.486
3,45.812,5.874


In [5]:
p = plot_user_loc(box, title="Bounding Box", color="navy", alpha=0.7, width=600, height=400, size=12)
show(p)

In [10]:
box = bounding_box(users, expand=0.005)
print(box)
p = plot_user_loc(box, title="Bounding Box", color="navy", alpha=0.7, width=600, height=400, size=12)
show(p)

   latitude  longitude
0    47.838     10.509
1    47.838      5.851
2    45.802      5.851
3    45.802     10.509


In [6]:
rect = pd.DataFrame([{"latitude": 45.74002, "longitude": 5.74975}, {"latitude": 47.79051, "longitude": 10.6232}])

p = plot_user_loc(rect, title="Bounding Rectangle", color="navy", alpha=0.7, width=600, height=400, size=12)
show(p)

In [6]:
user_data = load_user_gps_csv(5924, from_day_n=2, to_day_n=3)[["latitude", "longitude"]]
show(plot_user_loc(user_data, title="sample", width=600, height=400))

In [7]:
# user_data.to_csv("/home/tales/dev/map_matching/map_matching/switz_sample.csv", index=False)

### Spliting Box

In [20]:
import math
lat_amplitude = box["longitude"].max() - box["longitude"].min()

n_split = 6

for i in range(n_split):
    left = box["longitude"].min() + (i * (lat_amplitude / n_split))
    if (i + 1) == n_split:
        right = box["longitude"].max()
    else:
        right = box["longitude"].min() + ((i + 1) * (lat_amplitude / n_split))
    
    print(left, box["latitude"].min(), right, box["latitude"].max())
    

5.87402 45.8123 6.6426 47.828
6.6426 45.8123 7.41118 47.828
7.41118 45.8123 8.17976 47.828
8.17976 45.8123 8.94834 47.828
8.94834 45.8123 9.71692 47.828
9.71692 45.8123 10.4855 47.828
